In [2]:
# REQUIRED LIBARARIES

!pip install streamlit
!pip install pandas
!pip install joblib
!pip install dill



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%%writefile app.py
import streamlit as st
import pandas as pd
from joblib import load
import dill


Overwriting app.py


In [4]:
!pip install pandas

import pandas as pd
import numpy as np
df = pd.read_csv("Employee.csv")
df


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1
...,...,...,...,...,...,...,...,...,...
4648,Bachelors,2013,Bangalore,3,26,Female,No,4,0
4649,Masters,2013,Pune,2,37,Male,No,2,1
4650,Masters,2018,New Delhi,3,27,Male,No,5,1
4651,Bachelors,2012,Bangalore,3,30,Male,Yes,2,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4653 entries, 0 to 4652
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Education                  4653 non-null   object
 1   JoiningYear                4653 non-null   int64 
 2   City                       4653 non-null   object
 3   PaymentTier                4653 non-null   int64 
 4   Age                        4653 non-null   int64 
 5   Gender                     4653 non-null   object
 6   EverBenched                4653 non-null   object
 7   ExperienceInCurrentDomain  4653 non-null   int64 
 8   LeaveOrNot                 4653 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 327.3+ KB


In [6]:
# TRANSFORM LEAVEORNOT COLUMN (0 = no,1 = yes)

df["LeaveOrNot"]=df["LeaveOrNot"].map({1:"Yes",0:"No"})

X=df.drop(columns=["LeaveOrNot"])
y=df["LeaveOrNot"]


In [7]:
# CATEGORICAL FEATURES
categorical_features = X.select_dtypes(include=[object]).columns
categorical_features= list(categorical_features.difference(["LeaveOrNot"]))

print("Categorical Features:",categorical_features)

Categorical Features: ['City', 'Education', 'EverBenched', 'Gender']


In [8]:
# NUMERICAL FEATURES
numerical_features = list(X.select_dtypes(include=[np.float64,np.int64]))
print('Numerical Features:', numerical_features)

Numerical Features: ['JoiningYear', 'PaymentTier', 'Age', 'ExperienceInCurrentDomain']


In [10]:
# SPLIT TRAIN TEST DATA
from sklearn.model_selection import train_test_split

# SPLIT DATA INTO TRAIN AND TEST SETS
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('Train Data:','\n',"\n",y_train.value_counts(normalize=True),'\n',"\n",'Test Data:','\n', y_test.value_counts(normalize=True))

Train Data: 
 
 LeaveOrNot
No     0.656368
Yes    0.343632
Name: proportion, dtype: float64 
 
 Test Data: 
 LeaveOrNot
No     0.655209
Yes    0.344791
Name: proportion, dtype: float64


In [11]:
def summarize_cat(data,categorical_features):
  results=[]

  for column in data[categorical_features]:
      members = data[column].unique().tolist()
      results.append([column, members])
  return pd.DataFrame(results, columns=['Column Name', 'Members'])
summarize_cat(X_train,categorical_features)

,Column Name,Members
0,City,"[New Delhi, Bangalore, Pune]"
1,Education,"[Masters, Bachelors, PHD]"
2,EverBenched,"[No, Yes]"
3,Gender,"[Male, Female]"


In [12]:
summarize_cat(df,categorical_features).to_dict()

{'Column Name': {0: 'City', 1: 'Education', 2: 'EverBenched', 3: 'Gender'},
 'Members': {0: ['Bangalore', 'Pune', 'New Delhi'],
  1: ['Bachelors', 'Masters', 'PHD'],
  2: ['No', 'Yes'],
  3: ['Male', 'Female']}}

In [30]:
my_feature_dict = {'CATEGORICAL' : summarize_cat(df,categorical_features).to_dict(), 'NUMERICAL' : {'Column Name': numerical_features}}
my_feature_dict.get('NUMERICAL')

my_feature_dict

{'CATEGORICAL': {'Column Name': {0: 'City',
   1: 'Education',
   2: 'EverBenched',
   3: 'Gender'},
  'Members': {0: ['Bangalore', 'Pune', 'New Delhi'],
   1: ['Bachelors', 'Masters', 'PHD'],
   2: ['No', 'Yes'],
   3: ['Male', 'Female']}},
 'NUMERICAL': {'Column Name': ['JoiningYear',
   'PaymentTier',
   'Age',
   'ExperienceInCurrentDomain']}}

In [31]:
import pickle

# SAVING DICTIONARY TO MY_FEATURE_DICT.PKL FILE:

with open('my_feature_dict.pkl', 'wb') as fp:
    pickle.dump(my_feature_dict, fp)
    print('Dictionary is saved to my_feature_dict file')

Dictionary is saved to my_feature_dict file


In [34]:
# CREATE A PIPELINE
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

pipeline_num = Pipeline(steps=[
    ('scale_data', StandardScaler()),
    ('simple_imputer1', SimpleImputer(strategy='constant',fill_value=0)),
])

from sklearn.preprocessing import OneHotEncoder

pipeline_cat = Pipeline(steps=[
    ('OneHotEncode', OneHotEncoder(handle_unknown="ignore"))
])

from sklearn.compose import ColumnTransformer

preprocessor_stage_1 = ColumnTransformer(
    transformers=[
        ('cat', pipeline_cat, categorical_features), 
        ('num', pipeline_num, numerical_features),     
    ],remainder='drop')

preprocessor_stack = Pipeline(steps=[
    ('preprocessor_stage_1', preprocessor_stage_1)
])
preprocessor_stack



Pipeline(steps=[('preprocessor_stage_1',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('OneHotEncode',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['City', 'Education',
                                                   'EverBenched', 'Gender']),
                                                 ('num',
                                                  Pipeline(steps=[('scale_data',
                                                                   StandardScaler()),
                                                                  ('simple_imputer1',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant'))]),
                                                  ['JoiningYear', 'PaymentTier',
                                                   'Age',
                                                   'ExperienceInCurrentDomain'])]))])

In [35]:
preprocessor_stack.fit(X_train)

Pipeline(steps=[('preprocessor_stage_1',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('OneHotEncode',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['City', 'Education',
                                                   'EverBenched', 'Gender']),
                                                 ('num',
                                                  Pipeline(steps=[('scale_data',
                                                                   StandardScaler()),
                                                                  ('simple_imputer1',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant'))]),
                                                  ['JoiningYear', 'PaymentTier',
                                                   'Age',
                                                   'ExperienceInCurrentDomain'])]))])

In [36]:
pd.DataFrame(preprocessor_stack.transform(X_train),columns=preprocessor_stack[-1].get_feature_names_out())

,cat__City_Bangalore,cat__City_New Delhi,cat__City_Pune,cat__Education_Bachelors,cat__Education_Masters,cat__Education_PHD,cat__EverBenched_No,cat__EverBenched_Yes,cat__Gender_Female,cat__Gender_Male,num__JoiningYear,num__PaymentTier,num__Age,num__ExperienceInCurrentDomain
0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,-1.109740,0.539424,0.117526,-1.861550
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,-1.645675,0.539424,-0.914641,0.061305
2,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.034001,-1.226396,-0.088907,-0.579646
3,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,-1.645675,0.539424,-1.121074,-0.579646
4,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.034001,-1.226396,1.149693,-0.579646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3717,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,-0.573805,0.539424,0.530393,-1.220598
3718,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,-1.109740,0.539424,-0.708208,0.702257
3719,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.498065,0.539424,1.975427,-1.220598
3720,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,-0.037870,0.539424,2.388294,-1.220598


In [37]:
# RANDOM FOREST MODEL
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_stack),
    ('classifier', RandomForestClassifier())
])

# PIPELINE FITTING ON THE TRAINING DATA
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('preprocessor_stage_1',
                                  ColumnTransformer(transformers=[('cat',
                                                                   Pipeline(steps=[('OneHotEncode',
                                                                                    OneHotEncoder(handle_unknown='ignore'))]),
                                                                   ['City',
                                                                    'Education',
                                                                    'EverBenched',
                                                                    'Gender']),
                                                                  ('num',
                                                                   Pipeline(steps=[('scale_data',
                                                                                    StandardScaler()),
                                                                                   ('simple_imputer1',
                                                                                    SimpleImputer(fill_value=0,
                                                                                                  strategy='constant'))]),
                                                                   ['JoiningYear',
                                                                    'PaymentTier',
                                                                    'Age',
                                                                    'ExperienceInCurrentDomain'])]))])),
                ('classifier', RandomForestClassifier())])

In [38]:
# CHECK ACCURACY OF THE TRAINING MODEL
y_train_pred = pipeline.predict(X_train)
# Evaluating the model:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_train,y_train_pred))
print("\nClassification Report:\n", classification_report(y_train,y_train_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_train,y_train_pred))

Accuracy: 0.9279957012358947

Classification Report:
               precision    recall  f1-score   support

          No       0.92      0.98      0.95      2443
         Yes       0.95      0.84      0.89      1279

    accuracy                           0.93      3722
   macro avg       0.93      0.91      0.92      3722
weighted avg       0.93      0.93      0.93      3722


Confusion Matrix:
 [[2383   60]
 [ 208 1071]]


In [39]:
# TEST PREDICTION
my_pred_array=X_test.iloc[15:16:]
my_pred_array

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain
1128,Bachelors,2015,Pune,2,26,Female,No,4


In [40]:
pd.DataFrame(preprocessor_stack.transform(my_pred_array),columns=preprocessor_stack[0].get_feature_names_out())
##predicting:
y_pred = pipeline.predict(my_pred_array)
y_pred


array(['Yes'], dtype=object)

In [41]:
import dill
with open('pipeline.pkl', 'wb') as file:
    dill.dump(pipeline, file)

print('Pipeline saved  to file')

Pipeline saved  to file


##### STREAMLIT APP CODE STARTS HERE

In [42]:
%%writefile app.py
import streamlit as st
import pandas as pd
from joblib import load
import dill

#LOAD MODEL FEATURES
with open('pipeline.pkl', 'rb') as f:
    model=dill.load(f)

my_feature_dict=load('my_feature_dict.pkl')

#PREDICTION LOGIC
def predict_churn(input_df):
    return model.predict(input_df)

#APPLICATION USER INTERFACE
st.title("Employee Churn Predictor")
st.subheader("Based on Employee Dataset")
# CATEGORICAL FEATURES:
st.subheader("Categorical Details")
categorical_info = my_feature_dict.get('CATEGORICAL')
categorical_data = {}
for idx, feature in enumerate(categorical_info.get('Column Name').values()):
    options = categorical_info.get('Members')[idx]
    categorical_data[feature] = st.selectbox(f"{feature}", options, key=f"cat_{feature}")

# NUMERICAL FEATURES:
st.subheader("Job & Personal Information")
numerical_info = my_feature_dict.get('NUMERICAL')
numerical_data = {}

for feature in numerical_info.get('Column Name'):
    if feature == "Age":
        numerical_data[feature] = st.slider("Age", 18,70,18, key=f"num_{feature}")
    elif feature == "JoiningYear":
        numerical_data[feature] = st.number_input("Year of Joining", min_value=1992, max_value=2025, value=2018, key=f"num_{feature}")
    elif feature == "PaymentTier":
        numerical_data[feature] = st.radio("Payment Tier",[1, 2, 3],index=1, key=f"num_{feature}")
    elif feature == "ExperienceInCurrentDomain":
        numerical_data[feature] = st.slider("Experience (Years)",0,30,1, key=f"num_{feature}")
    else:
        numerical_data[feature] = st.number_input(f"{feature}",key=f"num_{feature}")
combined_input = {**categorical_data, **numerical_data}
input_df = pd.DataFrame([combined_input])
#PREDICTION
if st.button("Predict"):
    result = predict_churn(input_df)[0]
    meaning = {"Yes": "expected", "No": "not expected"}.get(result, "unknown")
    st.markdown(f"The model predicts the employee is *{meaning}* to churn.")
    
#CREATED BY
st.markdown("---")
st.caption("Created by Javeria Zakir")

Overwriting app.py
